# Оперирование таблицами `pandas`


In [ ]:
import pandas as pd
import os
path = os.path.join("..", "..", "assets", "data", "tables", "planets.csv")

planets = pd.read_csv(path, index_col="Название")


## apply, map, applymap

Применить некоторую функцию к всем строкам/столбцам таблицы можно методом [DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html). Передавать этому методу необходимо такую функцию, рассчитывая, что ей на вход будет подаваться объект `pd.series`, который в зависимости от значения параметра `axis` будет или строкой или столбцом таблицы.

In [1]:
import pandas as pd

def f(x):
    return x.max() - x.min() 

df.apply(f, axis="rows").head()

NameError: name 'df' is not defined

Метод [Series.map](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) позволяет применить функцию к каждому значению столбца. Чтобы продемонстрировать принцип работы этого метода, извлечем из столбца `brand` марку и модель автомобиля в разные столбцы.

Для этого определим функции, которые в качестве аргумента принимают строку из столбца `brand` и возвращают марку или модель автомобиля из неё. В данной таблице, марка автомобиля всегда встречается до первого пробела, а модель автомобиля --- все после первого пробела.

In [ ]:
def get_brand(x):
    return x.split(" ")[0]

def get_model(x):
    b, *m = x.split(" ") # все после первого пробела попадает в переменную m
    return " ".join(m)

x = "Mazda RX4 Wag"
print(f"{x} -> {get_brand(x)} and {get_model(x)}")

Определив эти функции применим их к столбцу `brand`. Т.к. `brand` --- индекс, то создадим из него `pd.Series`, применим определенные функции методом `map` и запишем результаты в таблицу в качестве новых столбцов.

In [ ]:
fullname = pd.Series(df.index, index=df.index)
df.index.rename("car", inplace=True)
df["brand"] = fullname.map(get_brand)
df["model"] = fullname.map(get_model)
df

Вместо функции, методу `map` можно передать словарь.

In [ ]:
n2str = {
    4.0: "four",
    6.0: "six",
    8.0: "eight"
    }
df["cyl"] = df["cyl"].map(n2str)
df

```{warning}
Использовать числа с плавающей точкой в качестве ключей словаря считается плохой практикой из-за особенностей их представления в компьютере: даже минимальное отклонение от точного значения ключа в словаре может привести (и почти наверняка приведет) к другому значению хэш-функции и получить искомое значение по "испорченному ключу не выйдет".
```

Метод [DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html#pandas.DataFrame.applymap) применяет функцию к кажому значению в таблице.

## Группировка по значениям столбца

Часто необходимо сгруппировать строки таблицы по какому-то принципу и с каждой из групп проделать какие-то операции. За группировку в `pandas` отвечает метод [DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html). В качестве аргумента её можно передать имя столбца, по значениям которого необходима группировка (можно также передать несколько столбцов в списке).

In [ ]:
grouped = df.groupby("brand")
grouped.get_group("Mazda")

Каждую группу можно рассматривать как мини-таблицу. Часто таблицу группируют по значениям одного столбца, чтобы потом посчитать какие-нибудь агрегирующие функцию к значениям других столбцов. Сделать это можно методом [DataFrameGroupBy.aggregate](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html). 

In [2]:
grouped["mpg"].aggregate(["min", "mean", "max", "count"])

NameError: name 'grouped' is not defined